# Fussing with the Garnett data products

In [29]:
# imports
import numpy as np

from pkg_resources import resource_filename

from astropy.table import Table, MaskedColumn

from specdb.cat_utils import match_ids

from dla_cnn import io as dla_io

read_sets: Using set file -- 
  /home/xavier/local/Python/linetools/linetools/lists/sets/llist_v1.2.ascii
Loading abundances from Asplund2009
Abundances are relative by number on a logarithmic scale with H=12


/home/xavier/local/Python/linetools/linetools/lists/linelist.py:402: RuntimeWarning: divide by zero encountered in log10
  self._extra_table['log(w*f)'] = np.log10(qm_strength)


## Check original tables

In [32]:
g16_abs = dla_io.load_garnett16(orig=True)

In [33]:
g16_abs.keys()

['THINGID',
 'SDSS',
 'Plate',
 'MJD',
 'Fiber',
 'RAdeg',
 'DEdeg',
 'z_QSO',
 'SNRSpec',
 'b_z_DLA',
 'B_z_DLA',
 'log.pnDLA',
 'log.pDLA',
 'log.pDnDLA',
 'log.pDDLA',
 'pnDLAD',
 'pDLAD',
 'z_DLA',
 'log.NHI',
 'flg_BAL',
 'SNR']

## Convert MNRAS tables to something useful

### Read

#### Table 1

In [34]:
tbl1_file = resource_filename('dla_cnn', 'catalogs/boss_dr12/MNRAS/table1.dat')

In [35]:
tbl1 = Table.read(tbl1_file, format='ascii')

In [36]:
tbl1[0:5]

THINGID,SDSS,Plate,MJD,Fiber,RAdeg,DECdeg,z_QSO,SNR,flg_BAL
int64,str18,int64,int64,int64,float64,float64,float64,float64,int64
268514930,000000.45+174625.4,6173,56238,528,0.0018983,17.7737391,2.3091,0.7795,0
245101352,000000.66+145828.8,6177,56268,595,0.0027564,14.9746755,2.4979,3.5009,0
154279486,000000.97+044947.2,4415,55831,464,0.0040524,4.8297806,1.6188,7.1102,1000
62145359,000001.27-020159.7,4354,55810,678,0.005317,-2.0332733,1.3604,6.1981,1000
66692618,000001.37-011930.0,4354,55810,646,0.0057462,-1.3250088,2.328,8.2577,0


#### Table 2

In [37]:
tbl2_file = resource_filename('dla_cnn', 'catalogs/boss_dr12/MNRAS/table2.dat')

In [38]:
tbl2 = Table.read(tbl2_file, format='ascii')

In [39]:
tbl2[0:5]

THINGID,zmin,zmax,log.pnDLA,log.pDLA,log.pDnDLA,log.pDDLA,pnDLAD,pDLAD,z_DLA,log.NHI
int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
268514930,1.9654,2.2989,-0.03081,-3.49537,-1043.59,-1042.56,0.919661,0.0803389,2.216,20.0077
245101352,1.9661,2.4877,-0.04589,-3.10426,-621.961,-609.072,5.37594e-05,0.999946,1.997,20.0544
66692618,1.9552,2.318,-0.03247,-3.44363,-345.637,-350.097,0.999618,0.000381529,1.997,20.0038
374107623,1.9328,2.3666,-0.03603,-3.34124,-1074.77,-1079.59,0.999703,0.000296901,1.9438,20.0021
237410936,2.544,3.7018,-0.10786,-2.28033,-579.993,-588.618,0.99998,2.04506e-05,2.8005,20.0045


In [40]:
len(tbl1), len(tbl2)

(297301, 162858)

### Match em up

In [14]:
match_ids?

In [41]:
rows = match_ids(tbl2['THINGID'], tbl1['THINGID'])

### Add the columns

In [44]:
for key in tbl2.keys():
    if key == 'THINGID':
        continue
    # 
    print('adding column {:s}'.format(key))
    new_clm = MaskedColumn(-999.*np.ones_like(tbl1['RAdeg']))
    new_clm[rows] = tbl2[key]
    # Mask it
    msk = new_clm == -999.
    new_clm.mask[msk] = True
    # Add it
    tbl1.add_column(new_clm, name=key)
    

adding column zmin
adding column zmax
adding column log.pnDLA
adding column log.pDLA
adding column log.pDnDLA
adding column log.pDDLA
adding column pnDLAD
adding column pDLAD
adding column z_DLA
adding column log.NHI


In [45]:
tbl1[0:5]

THINGID,SDSS,Plate,MJD,Fiber,RAdeg,DECdeg,z_QSO,SNR,flg_BAL,zmin,zmax,log.pnDLA,log.pDLA,log.pDnDLA,log.pDDLA,pnDLAD,pDLAD,z_DLA,log.NHI
int64,str18,int64,int64,int64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64
268514930,000000.45+174625.4,6173,56238,528,0.0018983,17.7737391,2.3091,0.7795,0,1.9654,2.2989,-0.03081,-3.49537,-1043.59,-1042.56,0.919661,0.0803389,2.216,20.0077
245101352,000000.66+145828.8,6177,56268,595,0.0027564,14.9746755,2.4979,3.5009,0,1.9661,2.4877,-0.04589,-3.10426,-621.961,-609.072,5.37594e-05,0.999946,1.997,20.0544
154279486,000000.97+044947.2,4415,55831,464,0.0040524,4.8297806,1.6188,7.1102,1000,--,--,--,--,--,--,--,--,--,--
62145359,000001.27-020159.7,4354,55810,678,0.005317,-2.0332733,1.3604,6.1981,1000,--,--,--,--,--,--,--,--,--,--
66692618,000001.37-011930.0,4354,55810,646,0.0057462,-1.3250088,2.328,8.2577,0,1.9552,2.318,-0.03247,-3.44363,-345.637,-350.097,0.999618,0.000381529,1.997,20.0038


### Write

In [46]:
merged_file = resource_filename('dla_cnn', 'catalogs/boss_dr12/MNRAS/merged_g16.fits')

In [47]:
tbl1.write(merged_file)